Текст для корпуса я собрала из разных текстов в интернете и какие-то предложения написала сама. Постаралась, чтобы в него входили различные слова с дефисом, фамилии на -ов и -их, аббревиатуры и сокращения, а так же достаточно нестандартные и относительно редкие слова вроде "жахнуть" и "хой", так как в определении их частей речи могут возникать ошибки или несоответствия.

In [1]:
korpus = '''
На прошлой неделе Вронских уехал в Тель-Авив.
Сказал, что номер у него очень хороший, даже диван-кровать есть.
Съезд избрал ЦК РСДРП в составе 3 чел.: С. И. Радченко, Б. Л. Эйдельман и А. И. Кремер.
Концерт Славы КПСС прошел на ура — он собрал 3 тыс. человек.
Светлый лагер — простой и тонкий тип пива.
Универсальный напиток, подходящий к любой ситуации.
Он имеет светлый цвет от бледно-желтого до золотистого.
Эль, как правило, более плотный, сладкий, алкогольный, часто "мощный" напиток с тонами фруктов.
А вот ламбик — это бельгийское пиво самопроизвольного брожения.
В 1926 году 22-летний Роберт Оппенгеймер учится у Патрика Блэкетта в Кавендишской лаборатории в Кембридже.
Главным героем сериала «Во все тяжкие» является преподаватель химии Уолтер Уайт, а также его альтер-эго, именуемое «Гейзенбергом», которое медленно пробуждалось в процессе криминальной деятельности химика и, впоследствии, приобрело статус основного «Я».
Центрального героя песни, «смешного карлика-шута», приводят во дворец, чтобы развеселить загрустившего короля, но в результате король в буквальном смысле помирает со смеху, а его подданные сходят с ума от смеха и остаются жить в шутовском «веселом царстве».
Покойники, которых научили кричать «хой», пугают крестьян и заставляют подростка танцевать пого.
Иванов жахнул меня по плечу, так что я аж вздрогнул.
Эта катавасия мне порядком поднадоела, и я решил удалиться.
На прошлой неделе Хветек участвовал в мюзикле "Три мушкетёра" и многих поразил своим выступлением.
У вас еще есть возможность посетить шоу с участием этого южнокорейского артиста, но продажи билетов скоро закончатся!
'''

Сама я разметила корпус опираясь на теги mystem, но с некоторыми упрощениями: я объединила SPRO, ADVPRO и APRO в тег PRO, так как у natasha нет разделения между разными местоимениями (зато стоит отметить, что там есть разделение между именами собственными и другими существительными, но это пришлось не учитывать, так как в mystem такого нет).

Разметим с помощью майстем и добавим разметку в словарь

In [2]:
from pymystem3 import Mystem

In [3]:
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [5]:
myst_result = m.analyze(korpus)
mystem_d = {}
for word in myst_result:
    if 'analysis' in list(word.keys()):
        mystem_d[word["text"]] = word['analysis'][0]['gr'].split('=')[0].split(',')[0]

Теперь разметим тот же текст с помощью пайморфи. К сожалению, тут придется токенизировать отдельно.

In [ ]:
!pip3 install pymorphy2

In [7]:
from pymorphy2 import MorphAnalyzer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
morph = MorphAnalyzer()

In [9]:
pymorphy_d = {}
for word in word_tokenize(korpus):
    if word not in  string.punctuation:
        lemma = word
        pos = morph.parse(word)[0].tag.POS
        pymorphy_d[lemma] = pos

Наконец, разметим текст с помощью natasha

In [10]:
!pip3 install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=281b1780de93825be2c0412be70f934ed63c495f3bbea88b0c164497257b062b
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [11]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [12]:
doc = Doc(korpus)

In [13]:
natasha_d = {}
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for word in doc.tokens:
    if word.pos != "PUNCT":
        natasha_d[word.text] = word.pos

Перенесем эти три словаря в файлы, чтобы было удобнее на них смотреть

In [14]:
with open("pymorphy.txt", "w", encoding="utf-8") as file:
    for word in list(pymorphy_d.keys()):
        file.write(str(word) + " " + str(pymorphy_d[word]) + '\n')

with open("mystem.txt", "w", encoding="utf-8") as file:
    for word in list(mystem_d.keys()):
        file.write(word + " " + mystem_d[word] + '\n')

with open("natasha.txt", "w", encoding="utf-8") as file:
    for word in list(natasha_d.keys()):
        file.write(word + " " + natasha_d[word] + '\n')



Приведем теги к одному виду, опираясь на тегсет майстема

In [15]:
mystem_d2 = {}
for word in list(mystem_d.keys()):
    if mystem_d[word] == "ADVPRO":
        mystem_d2[word] = "PRO"
    elif mystem_d[word] == "APRO":
        mystem_d2[word] = "PRO"
    elif mystem_d[word] == "SPRO":
        mystem_d2[word] = "PRO"
    else:
        mystem_d2[word] = mystem_d[word]

with open("mystem2.txt", "w", encoding="utf-8") as file:
    for word in list(mystem_d2.keys()):
        file.write(word + " " + mystem_d2[word] + '\n')

В пайморфи меняем теги на соответствующие в нашей разметке, разные формы глаголов обьединяем в "V", причастия я отнесла к прилагательным, а деепричастия к глаголам.

In [16]:
pymorphy_d2 = {}
for word in list(pymorphy_d.keys()):
    if pymorphy_d[word] == "NOUN":
        pymorphy_d2[word] = "S"
    elif pymorphy_d[word] == "ADJF":
        pymorphy_d2[word] = "A"
    elif pymorphy_d[word] == "ADJS":
        pymorphy_d2[word] = "A"
    elif pymorphy_d[word] == "COMP":
        pymorphy_d2[word] = "ADV"
    elif pymorphy_d[word] == "VERB":
        pymorphy_d2[word] = "V"
    elif pymorphy_d[word] == "INFN":
        pymorphy_d2[word] = "V"
    elif pymorphy_d[word] == "PRTF":
        pymorphy_d2[word] = "A"
    elif pymorphy_d[word] == "PRTS":
        pymorphy_d2[word] = "A"
    elif pymorphy_d[word] == "GRND":
        pymorphy_d2[word] = "V"
    elif pymorphy_d[word] == "NUMR":
        pymorphy_d2[word] = "NUM"
    elif pymorphy_d[word] == "ADVB":
        pymorphy_d2[word] = "ADV"
    elif pymorphy_d[word] == "NPRO":
        pymorphy_d2[word] = "PRO"
    elif pymorphy_d[word] == "PRED":
        pymorphy_d2[word] = "ADV"
    elif pymorphy_d[word] == "COMP":
        pymorphy_d2[word] = "ADV"
    elif pymorphy_d[word] == "PREP":
        pymorphy_d2[word] = "PR"
    elif pymorphy_d[word] == "PRCL":
        pymorphy_d2[word] = "PART"
    else:
        pymorphy_d2[word] = str(pymorphy_d[word])

with open("pymorphy2.txt", "w", encoding="utf-8") as file:
    for word in list(pymorphy_d2.keys()):
        file.write(word + " " + str(pymorphy_d2[word]) + '\n')

In [17]:
natasha_d2 = {}
for word in list(natasha_d.keys()):
    if natasha_d[word] == "NOUN":
        natasha_d2[word] = "S"
    elif natasha_d[word] == "ADJ":
        natasha_d2[word] = "A"
    elif natasha_d[word] == "PROPN":
        natasha_d2[word] = "S"
    elif natasha_d[word] == "VERB":
        natasha_d2[word] = "V"
    elif natasha_d[word] == "CCONJ":
        natasha_d2[word] = "CONJ"
    elif natasha_d[word] == "PRON":
        natasha_d2[word] = "PRO"
    elif natasha_d[word] == "DET":
        natasha_d2[word] = "A"
    elif natasha_d[word] == "ADP":
        natasha_d2[word] = "PR"
    else:
        natasha_d2[word] = natasha_d[word]

with open("natasha2.txt", "w", encoding="utf-8") as file:
    for word in list(natasha_d2.keys()):
        file.write(word + " " + str(natasha_d2[word]) + '\n')

Новую разметку я соранила в ноые файлы, чтобы можно было проверить, все ли поменялось так, как надо.

Посчитаем accuracy (разметку, сделанную предварительно мной, я добавила в файл gold.txt, оттуда мы достанем его прочитав файл)

In [19]:
gold_d = {}
with open("gold.txt", "r") as file:
    for line in file.readlines():
        gold_d[line.split()[0]] = line.split()[1]

In [20]:
gold_num = 233
mystem_right = 0
pymorphy_right = 0
natasha_right = 0

for word in list(gold_d.keys()):
    if word in list(mystem_d2.keys()):
        if mystem_d2[word] == gold_d[word]:
            mystem_right += 1

for word in list(gold_d.keys()):
    if str(word) in list(pymorphy_d2.keys()):
        if pymorphy_d2[word] == gold_d[word]:
            pymorphy_right += 1

for word in list(gold_d.keys()):
    if word in list(natasha_d2.keys()):
        if natasha_d2[word] == gold_d[word]:
            natasha_right += 1

print(f"mystem accuracy: {mystem_right/gold_num}")
print(f"pymorphy accuracy: {pymorphy_right/gold_num}")
print(f"natasha accuracy: {natasha_right/gold_num}")

mystem accuracy: 0.776824034334764
pymorphy accuracy: 0.7553648068669528
natasha accuracy: 0.7510729613733905


Экьюраси получилась самой большой для майстем.

Думаю, в прошлой домашке в словаре пригодились бы сочетания не + прилагательное и слишком + прилагательное. Наличие их в словаре помогло бы увеличить количество слов, характерных для отрицательных отзывов, так как, кажется, что они бы были более характерны для плохой оценки чего-то.

Для общего пополнения словаря же могли бы пригодиться сочетания прилагательное + существительное, так как разных персонажей, например, в отзывах могли бы называть по-разному.



Чанкер я написала для игрушечного примера

In [21]:
text = """
Это просто текcт для проверки, можно сказать проверочный текст.
Не хочу делать его слишком длинным, поэтому не буду много писать.
Он получается не интересным, зато поможет определить, не ошибается ли написанная программа.
А то я в слишком волнительном настроении по этому поводу.
"""

In [24]:
import string
n_gramms = []
result = m.analyze(text)

for word in result:
    ind = result.index(word)
    prev_ind = ind - 2
    if result[prev_ind]['text'] in string.punctuation:
        prev_ind = ind - 3
    prev =  result[prev_ind]

    if 'analysis' in list(word.keys()):
        pos = word['analysis'][0]['gr'].split('=')[0].split(',')[0]
        token = word['text']

        if 'analysis' in list(prev.keys()):
            pos_prev = prev['analysis'][0]['gr'].split('=')[0].split(',')[0]
            word_prev = prev['text']

            #достаем сочетания не + прилагательное
            if str(word_prev).lower() == "не":
                if str(pos) == "A":
                    n_gramms.append(f'{word_prev} {str(token)}')

            #достаем сочетания слишком + прилагательное
            if str(word_prev).lower() == "слишком":
                if str(pos) == "A":
                    n_gramms.append(f'{word_prev} {str(token)}')

            #достаем сочетания прилагательное + существительное
            if str(pos_prev) == "A":
                if str(pos) == "S":
                    n_gramms.append(f'{word_prev} {str(token)}')


print(n_gramms)


['проверочный текст', 'слишком длинным', 'не интересным', 'слишком волнительном', 'волнительном настроении']
